<a id="section_CART"></a> 
## Workshop Final DS Digital House

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns
pd.set_option('display.max_columns', None)
import missingno as msno
#from dataprep.eda import plot, plot_correlation, create_report, plot_missing

%matplotlib inline

## Importando os dados

In [ ]:
# importando bases de treino e teste
df_test = pd.read_csv(r"../DataSet/test.csv", low_memory = False)
df_train = pd.read_csv(r"../DataSet/train.csv", low_memory = False)

## Agrupando os datasets para limpar os dados

In [ ]:
# # incluindo colunas para sperar os dados da mesma maneira que estavam originalmente
# df_train['test'] = 0
# df_test['test'] = 1

# # incluindo coluna de score com nan nos dados de teste
# df_test['Credit_Score'] = np.nan

In [ ]:
# concatenando os datasets ja que eles tem as mesmas colunas

df_total = pd.concat([df_train, df_test], ignore_index = True)
df_orig = df_total.copy()

In [ ]:
print('dados de treino: ', df_train.shape)
print('dados de teste: ', df_test.shape)
print('todos os dados agrupados: ', df_total.shape)

## Colunas dataset:

* ID - Identificador de entrada
* Customer_ID - ID cliente
* Month - Mês do ano
* Name - nome do cliente
* Age - Idade Cliente
* SSN - Social Security Number (CPF no Brasil)
* Occupation - Ocupação do cliente
* Annual_Income - renda anual
* Monthly_Inhand_Salary - Salario mensal do cliente
* Num_Bank_Accounts - quantidade de contas em bancos
* Num_Credit_Card - quantidade de cartões de crédito
* Interest_Rate - taxa de juros cartão de crédito
* Num_of_Loan - Quantidade de empréstimos feitos no banco
* Type_of_Loan - tipo de empréstimo feito pelo cliente
* Delay_from_due_date - qtd. de dias de atraso pagamento cartão
* Num_of_Delayed_Payment - Média de pagamentos atrasado pelo cliente
* Changed_Credit_Limit - Variação percentual de limite do cartão de crédito
* Num_Credit_Inquiries - Quantidade de "cobranças" no cartão
* Credit_Mix - mix de crédito do cliente
* Outstanding_Debt - restante à ser pago da dívida
* Credit_Utilization_Ratio - Taxa de utlização do cartão de crédito
* Credit_History_Age - Tempo de histórico de crédito do cliente
* Payment_of_Min_Amount - Pagamento minimo
* Total_EMI_per_month - Pagamento fixo em dolares por mes
* Amount_invested_monthly - Quantidade de dinheiro investido pelo cliente mensalmente
* Payment_Behaviour - Comportamento de pagamento cliente
* Monthly_Balance - Saldo Mensal Cliente
* Credit_Score - Target, Pontuação de uso de crédito
* test - coluna utilizada para separar o dataset nos dados de treino e teste

## Checando os principais valores de algumas colunas

In [ ]:
colunas = df_total.columns

for coluna in colunas:
    print('Variavel: ', coluna)
    print(20*'-')
    print(df_total[coluna].value_counts(dropna=False))

### Observações

1. Colunas numéricas com "_" ok
    * Age,
    * Annual_Income,
    * Monthly_Inhand_Salary,
    * Num_Bank_Accounts,
    * Num_Credit_Card,
    * Interest_Rate
    * Num_of_Loan
    * Delay_from_due_date
    * Num_of_Delayed_Payment
    * Changed_Credit_Limit
    * Num_Credit_Inquiries
    * Outstanding_Debt
    * Credit_Utilization_Ratio
    * Total_EMI_per_month
    * Amount_invested_monthly
    * Monthly_Balance
2. SSN #F%$D@*&8 ok
3. Occupation _______ ok
4. Type_of_Loan - transformar em lista e indexar
5. Changed_Credit_Limit "_" -> NaN ok
6. Credit_Mix "_" -> NaN ok
7. Credit_History_Age Transformar em qtd. Meses
8. Payment_of_Min_Amount "NM" -> NaN ok
9. Payment_Behaviour "!@9#%8" -> NaN e transformar dado ok

## Ajustando os campos númericos que estão definidos como string por terem underlines em alguns registros

In [ ]:
# Campos númericos que estão como string - retirar underline dos numeros

colunas_ul = ['Age', 'Annual_Income', 'Num_of_Loan', 'Num_of_Delayed_Payment',
              'Changed_Credit_Limit', 'Outstanding_Debt', 'Amount_invested_monthly', 'Monthly_Balance']
for row in colunas_ul:
    df_total[row] = df_total[row].str.replace(r'_+', '')

## Data Wrangling

In [ ]:
# Removendo coluna ID, completamente inútil para a análise
df_total.drop(['ID'], axis = 1, inplace = True)

# removendo caracter estranho do SSN
df_total['SSN'].replace('#F%$D@*&8', np.NaN, inplace=True)

# removendo os underlines e colocando NaN na coluna Occupation
df_total['Occupation'].replace('_______', np.NaN, inplace=True)

df_total['Changed_Credit_Limit'].replace(['_', ''], np.NaN, inplace=True)

df_total['Credit_Mix'].replace('_', np.NaN, inplace=True)

df_total['Payment_of_Min_Amount'].replace('NM', np.NaN, inplace=True)

df_total['Payment_Behaviour'].replace('!@9#%8', np.NaN, inplace=True)

In [ ]:
# convertendo Credit_History_Age em quantidade de meses
def converter_mes(x):
    if pd.notnull(x):
        ano = int(x.split(' ')[0])
        mes = int(x.split(' ')[3])
        return (ano*12)+mes
    else:
        return x

df_total['Credit_History_age'] = df_total['Credit_History_Age'].apply(lambda x: converter_mes(x)).astype(float)

In [ ]:
df_total['Type_of_Loan_ajustado'] = df_total['Type_of_Loan'].replace("[abc]* and ", " ", regex=True)

In [ ]:
# Esta função define a quantidade % de dados faltantes
def plot_nas(df: pd.DataFrame):
    if df.isnull().sum().sum() != 0:
        na_df = (df.isnull().sum() / len(df)) * 100      
        na_df = na_df.drop(na_df[na_df == 0].index).sort_values(ascending=False)
        missing_data = pd.DataFrame({'Missing Ratio %' :na_df})
        missing_data.plot(kind = "barh")
        plt.title("% de dados faltantes")
        plt.show()
    else:
        print('No NAs found')
        
plot_nas(df_total)

In [ ]:
df_total.info()

In [ ]:
#criando dicionario e convertendo os dados

dicionario_conversao = {
    'Age': int,
    'Num_Bank_Accounts': int,
    'Num_Credit_Card': int,
    'Num_of_Loan': int,
    'Num_of_Delayed_Payment': int,
    'Annual_Income' : float,
    'Monthly_Inhand_Salary' : float,
    'Interest_Rate' : float,
    'Delay_from_due_date' : float,
    'Changed_Credit_Limit' : float,
    'Num_Credit_Inquiries' : float,
    'Outstanding_Debt' : float,
    'Credit_Utilization_Ratio' : float,
    'Changed_Credit_Limit' : float,
    'Amount_invested_monthly' : float,
    'Total_EMI_per_month' : float,
    'Num_of_Delayed_Payment' : float,
    'Monthly_Balance' : float,
    # 'ID' : object,
    'Customer_ID' : object,
    'Name' : object,
    'Month' : object,
    'SSN' : object,
    'Type_of_Loan' : object,
    'Occupation' : object,
    'Credit_Mix' : object,
    'Payment_of_Min_Amount' : object,
    'Payment_Behaviour' : object
    # 'test' : object
    }
# aplicando as type para variaveis

df_total = df_total.astype(dicionario_conversao)

In [ ]:
df_total.head()

### Data cleaning - Age

 * Limitando a idade dos clientes de 0 a 100

In [ ]:
# df_total = df_total[(df_total['Age'] <= 100) & (df_total['Age'] >= 0)]

# sns.boxplot(data=df_total, y='Age', x='test')

print(df_total.Age.describe())

### Concentração por tipo de ocupação

In [ ]:
sns.countplot(data=df_total, x = 'Occupation')
plt.xticks(rotation=90)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize = (12, 10))
sns.boxplot(data = df_total, y = 'Age', x = 'Occupation')
plt.xticks(rotation = 90)
plt.title('Distribuição Idade x Ocupação', size = 20)
plt.xlabel('Ocupação')
plt.ylabel('Idade')
plt.show()

In [ ]:
f, ax = plt.subplots(figsize = (12, 10))
sns.boxplot(data = df_total, y = 'Num_Bank_Accounts', x = 'Occupation')
plt.xticks(rotation = 90)
plt.title('Distribuição Idade x Ocupação', size = 20)
plt.xlabel('Ocupação')
plt.ylabel('Idade')
plt.show()
print(df_total.Num_Bank_Accounts.describe())

In [ ]:
f, ax = plt.subplots(figsize = (12, 10))
sns.boxplot(data = df_total, y = 'Delay_from_due_date', x = 'Occupation')
plt.xticks(rotation = 90)
plt.title('Distribuição dias de atraso x Ocupação', size = 20)
plt.xlabel('Ocupação')
plt.ylabel('dias de atraso')
plt.show()
print(df_total.Delay_from_due_date.describe())

In [ ]:
df_total.columns

In [ ]:
f, ax = plt.subplots(figsize = (12, 10))
sns.boxplot(data = df_total, y = 'Num_Credit_Card', x = 'Occupation')
plt.xticks(rotation = 90)
plt.title('Distribuição Qtd. Cartão de Crédito x Ocupação', size = 20)
plt.xlabel('Ocupação')
plt.ylabel('Qtd. Cartão de Crédito')
plt.show()
print(df_total.Num_Credit_Card.describe())

In [ ]:
f, ax = plt.subplots(figsize = (12, 10))
sns.boxplot(data = df_total, y = 'Num_Bank_Accounts', x = 'Occupation')
plt.xticks(rotation = 90)
plt.title('Distribuição Qtd. Contas x Ocupação', size = 20)
plt.xlabel('Ocupação')
plt.ylabel('Qtd. Contas')
plt.show()
print(df_total.Num_Bank_Accounts.describe())

In [ ]:
f, ax = plt.subplots(figsize = (12, 10))
sns.boxplot(data = df_total, y = 'Num_of_Loan', x = 'Occupation')
plt.xticks(rotation = 90)
plt.title('Distribuição Qtd. Empréstimos x Ocupação', size = 20)
plt.xlabel('Ocupação')
plt.ylabel('Qtd. Empréstimos')
plt.show()
print(df_total.Num_of_Loan.describe())

In [ ]:
"""
numCols = df_total.select_dtypes([np.number]).columns

for col in numCols:
    fig, ax = plt.subplots(1, 2, figsize = (8,8))
    sns.boxplot(data=df_total, y=col, x = 'Credit_Score', ax=ax[0]) #color = ['#f5190a', '#1e9e19', '#dede16'])
    #sns.scatterplot(data=df_total,x = 'Credit_Score', y = s = 100, y=col, ax=ax[1], color ='#ee1199')
    plt.show()
"""

In [ ]:
msno.bar(df_total)

In [ ]:
sns.factorplot('Credit_Score', col = 'Occupation', data = df_total, kind = 'count', col_wrap = 4)

In [ ]:
grid = sns.FacetGrid(df_total, col = 'Credit_Score')
grid.map(sns.distplot, 'Monthly_Inhand_Salary')

In [ ]:
label = df_total.Credit_Score.value_counts().index
label_count = df_total.Credit_Score.value_counts().values
plt.figure(figsize=(3,3))
plt.pie(data=df_total, x=label_count, labels=label, autopct='%.2f', shadow=True, radius=1.5)
plt.title("Credit Score Pie Chart")
plt.show()

In [ ]:
#plot(df_total,'Occupation','Credit_Score')

In [ ]:
#plot(df_total,'Age','Credit_Score')

In [ ]:
plt.figure(figsize=(17,8))
sns.heatmap(df_total.corr(),annot=True,cmap='viridis')

In [ ]:
plt.figure(figsize=(25,6))
sns.violinplot(x='Payment_Behaviour',y='Age',data=df_total, hue='Credit_Score', palette='rainbow')
plt.title("Violin Plot of Payment Behaviour by Age, Separated by Credit Score")

In [ ]:
plt.figure(figsize=(25,6))
sns.violinplot(x='Payment_Behaviour',y='Credit_Utilization_Ratio',data=df_total, hue='Credit_Score', palette='rainbow')
plt.title("Violin Plot of Payment Behaviour by Credit Utilization Ratio, Separated by Credit Score")

In [ ]:
plt.figure(figsize=(59,1))
sns.displot(data=df_total, x="Credit_Utilization_Ratio", kde=True)

In [ ]:
numCols = ['Monthly_Inhand_Salary', 'Delay_from_due_date', 'Credit_Utilization_Ratio']

for col in numCols:
    plt.figure(figsize=(180,6))
    sns.displot(x=col,data=df_total, hue='Credit_Score', palette=["#ff006e", "#83c5be", "#3a0ca3"])
    plt.show()

In [ ]:
#DataPrep at its finest pra visualizar tudo, tirar insights, as distribuições, missing data zeros, etc e etc
# create_report(df_total)

In [ ]:
# convertendo mes para encoding

import datetime

df_total['Month'] = df_total['Month'].apply(lambda x: datetime.datetime.strptime(x, '%B').month)

In [ ]:
print(df_total.shape, df_total.isna().sum())

In [ ]:
df_nonull = df_total.dropna()

In [ ]:
# pegando indices

df2 = df_nonull.groupby(["Customer_ID"])["Month"].nlargest(1)

# list comprehension para juntar os índices

indice_final = [i[1] for i in df2.index.values]

# aplicando máscara, pegando somente linha do último mês de cada usuário

df_nonull_uniqueCID = df_total.loc[indice_final]

In [ ]:
sns.countplot(data=df_nonull_uniqueCID, x = 'Occupation')
plt.xticks(rotation=90)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize = (12, 10))
sns.boxplot(data = df_nonull_uniqueCID, y = 'Age', x = 'Occupation')
plt.xticks(rotation = 90)
plt.title('Distribuição Idade x Ocupação', size = 20)
plt.xlabel('Ocupação')
plt.ylabel('Idade')
plt.show()

In [ ]:
f, ax = plt.subplots(figsize = (12, 10))
sns.boxplot(data = df_nonull_uniqueCID, y = 'Num_Bank_Accounts', x = 'Occupation')
plt.xticks(rotation = 90)
plt.title('Distribuição Idade x Ocupação', size = 20)
plt.xlabel('Ocupação')
plt.ylabel('Idade')
plt.show()
print(df_total.Num_Bank_Accounts.describe())

In [ ]:
# aplicando IQR para as colunas Num_of_Loan, Num_Bank_Accounts, Num_Credit_Card, Delay_from_due_date
# a coluna age sera tratada limitando idades entre 0 e 100 anos

# age

testando_df = df_nonull_uniqueCID.copy()

print(testando_df.shape)

indice_age = testando_df[(testando_df['Age'] <= 18) | (testando_df['Age'] >= 100)].index

testando_df.drop(indice_age, inplace=True)

# Num of loan

q3, q1 = np.percentile(testando_df['Num_of_Loan'], [75, 25])

interquartil = q3 - q1

limsup = q3 + 1.5 * interquartil

testando_df = testando_df.drop(testando_df[(testando_df['Num_of_Loan'] > limsup) | (testando_df['Num_of_Loan'] <= 0)].index)

# Num_Bank_Accounts

q3, q1 = np.percentile(testando_df['Num_Bank_Accounts'], [75, 25])

interquartil = q3 - q1

limsup = q3 + 1.5 * interquartil

testando_df = testando_df.drop(testando_df[(testando_df['Num_Bank_Accounts'] > limsup) | (testando_df['Num_Bank_Accounts'] <= 0)].index)

# Num_Credit_Card

q3, q1 = np.percentile(testando_df['Num_Credit_Card'], [75, 25])

interquartil = q3 - q1

limsup = q3 + 1.5 * interquartil

testando_df = testando_df.drop(testando_df[(testando_df['Num_Credit_Card'] > limsup) | (testando_df['Num_Credit_Card'] <= 0)].index)

#Delay_from_due_date

q3, q1 = np.percentile(testando_df['Delay_from_due_date'], [75, 25])

interquartil = q3 - q1

limsup = q3 + 1.5 * interquartil

testando_df = testando_df.drop(testando_df[(testando_df['Delay_from_due_date'] > limsup) | (testando_df['Delay_from_due_date'] <= 0)].index)

testando_df.shape

In [ ]:
f, ax = plt.subplots(figsize = (12, 10))
sns.boxplot(data = testando_df, y = 'Delay_from_due_date', x = 'Occupation')
plt.xticks(rotation = 90)
plt.title('Distribuição dias de atraso x Ocupação', size = 20)
plt.xlabel('Ocupação')
plt.ylabel('dias de atraso')
plt.show()
print(df_total.Delay_from_due_date.describe())

In [ ]:
f, ax = plt.subplots(figsize = (12, 10))
sns.boxplot(data = testando_df, y = 'Num_Credit_Card', x = 'Occupation')
plt.xticks(rotation = 90)
plt.title('Distribuição Qtd. Cartão de Crédito x Ocupação', size = 20)
plt.xlabel('Ocupação')
plt.ylabel('Qtd. Cartão de Crédito')
plt.show()
print(df_total.Num_Credit_Card.describe())

In [ ]:
f, ax = plt.subplots(figsize = (12, 10))
sns.boxplot(data = testando_df, y = 'Num_Bank_Accounts', x = 'Occupation')
plt.xticks(rotation = 90)
plt.title('Distribuição Qtd. Contas x Ocupação', size = 20)
plt.xlabel('Ocupação')
plt.ylabel('Qtd. Contas')
plt.show()
print(df_total.Num_Bank_Accounts.describe())

In [ ]:
f, ax = plt.subplots(figsize = (12, 10))
sns.boxplot(data = testando_df, y = 'Num_of_Loan', x = 'Occupation')
plt.xticks(rotation = 90)
plt.title('Distribuição Qtd. Empréstimos x Ocupação', size = 20)
plt.xlabel('Ocupação')
plt.ylabel('Qtd. Empréstimos')
plt.show()
print(df_total.Num_of_Loan.describe())

In [ ]:
# funcao para analisar IQR de colunas
def remocao_iqr(x):
    q3, q1 = np.percentile(x, [75, 25])

    interquartil = q3 - q1
    
    limsup = q3 + 1.5 * interquartil

### Pré-processamento para modelagem

In [ ]:
# Pré-processamento para modelos para encontrar melhores features

from sklearn.model_selection import train_test_split
import statsmodels.api as sm

df_processado = df_nonull_uniqueCID.copy()
df_processado = df_processado.drop(['Customer_ID', 'Name', 'SSN', 'Type_of_Loan', 'Type_of_Loan_ajustado',
                                    'Credit_History_Age', 'Credit_Score', 'Month', 'Monthly_Balance'], axis=1)

#Pegando variáveis categóricas e numéricas

categorical = list(df_processado.select_dtypes(include=['object']).columns)  #Talvez criar no começo pra visualizaçao describe().transpose()
numerical = list(df_processado.select_dtypes(include=['int64', 'float64']).columns) #Talvez criar no começo pra visualizaçao describe()

# Dummy Encoder para variáveis categóricas

df_processado_categoricals = pd.get_dummies(df_processado[categorical], drop_first=True)

#Scaling para variáveis numéricas

from sklearn.preprocessing import StandardScaler
stdscaler = StandardScaler()

df_processado_numericals = pd.DataFrame(stdscaler.fit_transform(df_processado[numerical]), columns = numerical, index = df_processado_categoricals.index)

#Concatenando categóricas encodadas e numéricas escaladas
df_processado_final = pd.concat([df_processado_numericals, df_processado_categoricals], axis=1)

#Ordinal Encoder na target
from sklearn.preprocessing import OrdinalEncoder
ordenc = OrdinalEncoder()

df_processado_final['Credit_Score'] = ordenc.fit_transform(df_nonull_uniqueCID['Credit_Score'].values.reshape(-1,1)).astype(int)

#Splitando o dataframe processado
X_train, X_test, y_train, y_test = train_test_split(df_processado_final.drop('Credit_Score', axis=1), df_processado_final['Credit_Score'], test_size=0.3, random_state=101)

In [ ]:
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

df_processado = df_nonull_uniqueCID.copy()
df_processado = df_processado.drop(['Customer_ID', 'Name', 'SSN', 'Type_of_Loan', 'Type_of_Loan_ajustado',
                                    'Credit_History_Age', 'Credit_Score', 'Month', 'Monthly_Balance'], axis=1)

#Pegando variáveis categóricas e numéricas

categorical = list(df_processado.select_dtypes(include=['object']).columns)  #Talvez criar no começo pra visualizaçao describe().transpose()
numerical = list(df_processado.select_dtypes(include=['int64', 'float64']).columns) #Talvez criar no começo pra visualizaçao describe()

# Dummy Encoder para variáveis categóricas

df_processado_categoricals = pd.get_dummies(df_processado[categorical], drop_first=True)

#Scaling para variáveis numéricas

from sklearn.preprocessing import StandardScaler
stdscaler = StandardScaler()

df_processado_numericals = pd.DataFrame(stdscaler.fit_transform(df_processado[numerical]), columns = numerical, index = df_processado_categoricals.index)

#Concatenando categóricas encodadas e numéricas escaladas
df_processado_final = pd.concat([df_processado_numericals, df_processado_categoricals], axis=1)

#Ordinal Encoder na target
from sklearn.preprocessing import OrdinalEncoder
ordenc = OrdinalEncoder()

df_processado_final['Credit_Score'] = ordenc.fit_transform(df_nonull_uniqueCID['Credit_Score'].values.reshape(-1,1)).astype(int)

#Splitando o dataframe processado
X_train, X_test, y_train, y_test = train_test_split(df_processado_final.drop('Credit_Score', axis=1), df_processado_final['Credit_Score'], test_size=0.3, random_state=101)

### Seleção de Features e Feature Importance

In [ ]:
#Dropar colunas inúteis (ids etc) e Changed Credit Limit e Credi History Age (vide heatmap)

#ExtraTreeClassifier para usar método feature_importances_

#Logistic regression do statsmodels para usar os testes de hipótese do método summary()

#Técnicas de feature selection da última aula sobre o tema

#Checar variância de cada feature
# pd.options.display.float_format = '{:.6f}'.format
# df_heart.apply(np.var).sort_values(ascending = False)[ : 8]

In [ ]:
# Modelo Statsmodels para teste de hipótese e seleção de features

# X_train = sm.add_constant(X_train)
# X_test = sm.add_constant(X_test)

model = sm.OLS(y_train, X_train)
non_reg_OLS = model.fit()
non_reg_OLS.summary()

In [ ]:
#VIF
def calc_vif(data):
    vif_df = pd.DataFrame(columns=['Var', 'VIF'])
    x_var_names = data.columns
    
    for i in range(0, x_var_names.shape[0]):
        y = data[x_var_names[i]]
        x = data[x_var_names.drop(x_var_names[i])]
        r2 = sm.OLS(y, x).fit().rsquared
        vif = round(1/(1-r2),3)
        vif_df.loc[i] = [x_var_names[i], vif]
    return vif_df.sort_values(by='VIF',axis = 0, ascending=False, inplace=False)

calc_vif(X_train)

In [ ]:
#ExtraTreeClassifier para feature_importances_

from sklearn.tree import ExtraTreeClassifier

xtc = ExtraTreeClassifier()
xtc.fit(X_train, y_train)
feat_importance = pd.Series(xtc.feature_importances_, index=X_train.columns).sort_values(ascending=False)

plt.figure(figsize=(20,8))
feat_importance.plot(kind='bar')

### PCA - Visualização e Feature Importance

In [ ]:
# Visualização da Análise de Componentes Principais (PCA)
from sklearn.decomposition import PCA

pca = PCA().fit(X_train)

plt.plot(pca.explained_variance_ratio_.cumsum(), lw=3, color='#087E8B')
plt.axhline(0.7, ls='--', color='k')
plt.axvline(9, ls='--', color='k')
plt.text(9.9, 0.72, '70% da variância', fontsize=14)
plt.text(9.8, 0.4, '9 componentes', fontsize=14)
plt.title('Variância cumulativa explicada por número de componentes principais', size=18)
plt.show()

In [ ]:
print('Número de componentes principais para explicar pelo menos 70% da variância: {}'.format(np.argmax(pca.explained_variance_ratio_.cumsum() > 0.7)))
print(f'9 Componentes principais explicam:{pca.explained_variance_ratio_[:10].sum().round(4)*100}% da variância')

In [ ]:
loadings = pd.DataFrame(
    data=pca.components_.T * np.sqrt(pca.explained_variance_), 
    columns=[f'PC{i}' for i in range(1, len(X_train.columns) + 1)],
    index=X_train.columns
)

plt.figure(figsize=(30,18))
sns.heatmap(loadings.iloc[:,:10], annot=True, cmap='viridis')

In [ ]:
pc1_loadings = loadings.sort_values(by='PC1', ascending=False)[['PC1']]
pc1_loadings = pc1_loadings.reset_index()
pc1_loadings.columns = ['Attribute', 'CorrelationWithPC1']

plt.bar(x=pc1_loadings['Attribute'], height=pc1_loadings['CorrelationWithPC1'], color='#087E8B')
plt.title('PCA loading scores (first principal component)', size=20)
plt.xticks(rotation='vertical')
plt.show()

### Meme do homem-aranha apontando pro homem-aranha

##### - Inputação de Dados

In [ ]:
# escolher colunas para inputar e as formas de fazê-lo


### Modelagem

In [ ]:
# modelos referência (random ou dummy classifier),
# base (random forest com plot, extratrees, XGBoost, rede neural simples)
# e ensembles (stacking de xgboost com PCA)